# <a id='Урок_12'>Домашнее задание №12. Модель Transformer-2 (Text Summarization)</a>

## <a id='Задание_12'>Задание</a>

1. Реализовать суммаризацию текста
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
Проверить насколько хорошо она суммаризирует
2. (дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

## <a id='Подготовка_12'>Подготовка</a>

In [1]:
#!g1.1
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GRU, AdditiveAttention
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, T5ForConditionalGeneration
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
import unicodedata
import re
import numpy as np
import os
import io
import time

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## <a id='Загрузка_данных_12'>Загрузка данных</a>

In [2]:
#!g1.1
# Загрузка датасета
dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")

No config specified, defaulting to: gazeta/default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /tmp/xdg_cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
#!g1.1
# Структура датасета
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 52400
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5770
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 5265
    })
})

In [4]:
#!g1.1
# Тренировочный датасет
import pandas as pd
df_train = pd.DataFrame(dataset['train'])
df_train.head()

,text,summary,title,date,url
0,«По итогам 2011 года чистый отток может состав...,"В 2011 году из России уйдет $80 млрд, считают ...",Прогноз не успевает за оттоком,2011-11-30 18:33:39,https://www.gazeta.ru/financial/2011/11/30/385...
1,Российское подразделение интернет-корпорации G...,"Юлия Соловьева, экс-директор холдинга «Профмед...",Google закончил поиск,2013-01-24 18:20:09,https://www.gazeta.ru/business/2013/01/24/4939...
2,Басманный районный суд Москвы вечером 6 феврал...,Суд арестовал на два месяца четверых экс-чинов...,«Фигуранты дела могут давить на свидетелей»,2018-02-06 21:21:14,https://www.gazeta.ru/social/2018/02/06/116393...
3,Как повлияло вступление в ВТО на конкурентносп...,Мнения предпринимателей по поводу вступления в...,«С последних традиционно «отжимают» больше»,2013-06-21 17:43:50,https://www.gazeta.ru/business/2013/06/21/5388...
4,К третьему сезону «Голос» на Первом канале ста...,На Первом канале завершился третий сезон шоу «...,Третий «Голос» за Градского,2014-12-27 01:10:01,https://www.gazeta.ru/culture/2014/12/27/a_636...


In [5]:
#!g1.1
# Валидационный датасет
df_val = pd.DataFrame(dataset['validation'])
df_val.head()

,text,summary,title,date,url
0,"После громких приобретений Андре Шюррле, Гуса ...",Московский «Спартак» продолжает активную транс...,Замена Фернандо: кого может купить «Спартак»,2019-08-11 23:14:58,https://www.gazeta.ru/sport/2019/08/11/a_12572...
1,Американское издание The National Interest оце...,Издание The National Interest оценило перспект...,Шестое поколение: в США оценили российский бом...,2019-08-11 21:16:04,https://www.gazeta.ru/army/2019/08/11/12572317...
2,Министр иностранных дел России Сергей Лавров с...,"Глава МИД России Сергей Лавров заявил, что в 2...",«Зачем гонять людей?»: США требуют новый рефер...,2019-06-06 09:47:35,https://www.gazeta.ru/politics/2019/06/06_a_12...
3,Минфин предложил с января 2020 года увеличить ...,Министерство финансов предлагает вдвое поднять...,Перейти на отечественное: что будет с зарубежн...,2019-07-24 20:38:15,https://www.gazeta.ru/business/2019/07/24/1252...
4,Заявление командующего военно-воздушными силам...,Американские ПВО провалились в Саудовской Арав...,«Глупое заявление»: Медведев напомнил о провал...,2019-09-20 21:16:22,https://www.gazeta.ru/army/2019/09/20/12664933...


In [6]:
#!g1.1
# Тестовый датасет
df_test = pd.DataFrame(dataset['test'])
df_test.head()

,text,summary,title,date,url
0,Американское аэрокосмическое агентство NASA ог...,"В NASA назвали четыре миссии в дальний космос,...","Венера, Ио или Тритон: куда полетит NASA",2020-02-14 16:39:11,https://www.gazeta.ru/science/2020/02/14_a_129...
1,Около 11 тысяч зрителей увидели все самое лучш...,25 и 26 февраля в Кремлевском дворце съездов п...,«Люди в Бурятии очень талантливые»,2020-02-28 10:44:13,https://www.gazeta.ru/social/2020/02/28/129806...
2,7 ноября в Белоруссии прошли выборы членов сов...,В Белоруссии в день годовщины Октябрьской рево...,Вспомнить СССР: как Лукашенко провел выборы,2019-11-07 19:55:08,https://www.gazeta.ru/politics/2019/11/07_a_12...
3,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,2020-03-01 16:50:06,https://www.gazeta.ru/culture/2020/03/01/a_129...
4,Депутат Верховной рады от партии «Слуга народа...,Украина не должна выплачивать пенсии жителям Д...,«Поддерживают Россию»: почему Киев не платит п...,2020-02-06 12:41:24,https://www.gazeta.ru/business/2020/02/06/1294...


## <a id='Предобученная_модель_12'>Предобученная модель summarization IlyaGusev/rut5_base_sum_gazeta</a>

In [7]:
#!g1.1
# Загрузка и инициализация модели
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Пример работы модели
article_text = df_test['text'][1]

input_ids = tokenizer(
    [article_text],
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

В Кремле прошло праздничное шоу «Танцуют все!» на телеканале «Россия». Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов и эвенков.


In [8]:
#!g1.1
# Функция подсчета метрик
def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [9]:
#!g1.1
# Подсчет метрик на примере предсказания модели
target = sent_tokenize(df_test['summary'][1])
preds = sent_tokenize(summary)
calc_scores(target, preds)

Count: 2
Ref: Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.
Hyp: Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов и эвенков.
BLEU:  0.19334823151545613
ROUGE:  {'rouge-1': {'r': 0.029411764705882353, 'p': 0.03125, 'f': 0.030303027805326194}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.029411764705882353, 'p': 0.03125, 'f': 0.030303027805326194}}


**Вывод:** результаты модели довольно слабые.

## <a id='Обучение_модели_12'>Обучение модели для генерации заголовков</a>

In [10]:
#!g1.1
# Добавляем токены начала и конца предложения
df_train['title_clean'] = df_train['title'].apply(lambda v: 'BOS ' + v + ' EOS')
df_test['title_clean'] = df_test['title'].apply(lambda v: 'BOS ' + v + ' EOS')

In [11]:
#!g1.1
df_test['title_clean']

0        BOS Венера, Ио или Тритон: куда полетит NASA EOS
1              BOS «Люди в Бурятии очень талантливые» EOS
2       BOS Вспомнить СССР: как Лукашенко провел выбор...
3       BOS «Он очень переживал»: Бабкина об отношения...
4       BOS «Поддерживают Россию»: почему Киев не плат...
                              ...                        
5765    BOS Новая «Игра престолов»? Netflix показал тр...
5766    BOS «Это было ужасно»: Джоли эвакуировали со с...
5767    BOS «Это камни»: ученого затравили за змей на ...
5768    BOS «До 200 тысяч зараженных»: как распростран...
5769    BOS С российским реактором: Китай создаст атом...
Name: title_clean, Length: 5770, dtype: object

In [12]:
#!g1.1
# Уменьшаем объем датасетов
MAX_TRAIN_SAMPLE = 1000
MAX_TEST_SAMPLE = 200

df_train = df_train[:MAX_TRAIN_SAMPLE]
df_test = df_test[:MAX_TEST_SAMPLE]

In [13]:
#!g1.1
# Создаем словарь
max_len_text = 700
max_len_sum = 50

In [14]:
#!g1.1
# Преобразуем тексты
tok_text = Tokenizer()
# Создаем словарь индексов на основе частоты слов в df_train['text'] (чем меньше число, тем чаще встречается слово)
tok_text.fit_on_texts(df_train['text'])
# Преобразуем каждый text из df_train['text'] в последовательность целых чисел  в соответсвии со словарем
x_train_tok = tok_text.texts_to_sequences(df_train['text'])
x_test_tok = tok_text.texts_to_sequences(df_test['text'])

In [15]:
#!g1.1
text_vocab_size=len(tok_text.word_index)+1
# Приводим последовательности целых чисел к одной длине:
# К каждому элементу из x_train_tok добавляем (отнимаем) до max_len_text (от max_len_text) нулями сзади
padded_x_train = pad_sequences(x_train_tok, maxlen=max_len_text, padding='post', truncating='post')
padded_x_test = pad_sequences(x_test_tok, maxlen=max_len_text, padding='post', truncating='post')
padded_x_train.shape

(1000, 700)

In [16]:
#!g1.1
# Преобразуем заголовки
tok_sum = Tokenizer()
# Создаем словарь индексов на основе частоты слов в df_train['title_clean']
tok_sum.fit_on_texts(df_train['title_clean'])
# Преобразуем каждый text из df_train['title_clean'] в последовательность целых чисел  в соответсвии со словарем
x_train_sum = tok_sum.texts_to_sequences(df_train['title_clean'])
x_test_sum = tok_sum.texts_to_sequences(df_test['title_clean'])

In [17]:
#!g1.1
sum_vocab_size=len(tok_sum.word_index)+1
# Приводим последовательности целых чисел к одной длине:
# К каждому элементу из x_train_tok добавляем (отнимаем) до max_len_sum (от max_len_sum) нулями сзади
padded_x_train_sum = pad_sequences(x_train_sum, maxlen=max_len_sum, padding='post', truncating='post')
padded_x_test_sum = pad_sequences(x_test_sum, maxlen=max_len_sum, padding='post', truncating='post')
padded_x_train_sum.shape

(1000, 50)

In [18]:
#!g1.1
# Словарь индекс: токен для tok_text
reverse_text_index=tok_text.index_word
# Словарь индекс: токен для tok_sum
reverse_sum_index=tok_sum.index_word
# Словарь токен: индекс для tok_sum
sum_wordindex=tok_sum.word_index

**Построение и обучение модели**

In [19]:
#!g1.1
BUFFER_SIZE = len(padded_x_train)
BATCH_SIZE = 32
steps_per_epoch = len(padded_x_train)//BATCH_SIZE

# Каждую пару (padded_x_train, padded_x_train_sum) преобразуем в тензор
dataset = tf.data.Dataset.from_tensor_slices((padded_x_train, padded_x_train_sum)).shuffle(BUFFER_SIZE)
# Объединяем последоваетльные элементы в батчи, неполный батч удаляем
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

2023-03-05 16:54:21.088601: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 16:54:21.713100: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30995 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:8c:00.0, compute capability: 7.0


In [20]:
#!g1.1
for element in dataset:
  print(element)
  break

(<tf.Tensor: shape=(32, 700), dtype=int32, numpy=
array([[    1,   350,     1, ..., 13505,   203, 86528],
       [  363,  4844,   413, ...,     0,     0,     0],
       [  363,   198, 11394, ...,  1749,  1654,    20],
       ...,
       [  170,     1, 10562, ...,     0,     0,     0],
       [  299,  1341, 37457, ...,     0,     0,     0],
       [24241,  4485,  1220, ...,     0,     0,     0]], dtype=int32)>, <tf.Tensor: shape=(32, 50), dtype=int32, numpy=
array([[   1, 3256, 3257, ...,    0,    0,    0],
       [   1,   25, 1477, ...,    0,    0,    0],
       [   1, 3383,  401, ...,    0,    0,    0],
       ...,
       [   1,  903,    6, ...,    0,    0,    0],
       [   1,  424,  425, ...,    0,    0,    0],
       [   1, 2849, 2850, ...,    0,    0,    0]], dtype=int32)>)


In [21]:
#!g1.1
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 700]), TensorShape([32, 50]))

In [22]:
#!g1.1
# Построение модели (Encoder и Decoder)
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru1 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru1(x, initial_state = hidden)
        output, state = self.gru2(output, initial_state = state)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))
    
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = tf.keras.layers.AdditiveAttention()

    def call(self, x, query, value):
        # enc_output shape == (batch_size, max_length, hidden_size)
        #attention_weights = self.attention([ tf.expand_dims(query, 1), value,])
        context_vector = self.attention([tf.expand_dims(query, 1), value,])
        #context_vector = tf.squeeze(context_vector)


        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([context_vector, x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [23]:
#!g1.1
latent_dim = 300
embedding_dim=200

#создаем экземпляры encoder и decoder
encoder = Encoder(text_vocab_size, embedding_dim, latent_dim, BATCH_SIZE)
decoder = Decoder(sum_vocab_size, embedding_dim, latent_dim, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

2023-03-05 16:54:25.058427: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101


In [24]:
#!g1.1
sample_output

<tf.Tensor: shape=(32, 700, 300), dtype=float32, numpy=
array([[[-8.87439679e-03, -4.31183213e-03, -8.34321324e-03, ...,
          6.24656305e-03,  3.97082232e-03, -1.77751742e-02],
        [-1.08014718e-02, -2.99269916e-03, -7.72377476e-03, ...,
          9.91443545e-03,  1.12279905e-02, -7.43651763e-03],
        [-5.52456593e-03,  3.32956156e-03,  4.95634088e-03, ...,
          7.49652879e-03,  1.57716330e-02, -4.06223116e-03],
        ...,
        [-5.70216333e-04, -6.43045316e-03, -1.39458483e-04, ...,
         -1.46250811e-03,  5.61242504e-03,  1.89076643e-02],
        [-5.70216333e-04, -6.43045316e-03, -1.39458498e-04, ...,
         -1.46250811e-03,  5.61242504e-03,  1.89076643e-02],
        [-5.70216333e-04, -6.43045316e-03, -1.39458396e-04, ...,
         -1.46250811e-03,  5.61242504e-03,  1.89076643e-02]],

       [[-6.46381127e-03,  5.51322519e-05, -1.30271846e-02, ...,
          1.00127459e-02,  4.93167713e-03, -1.43956896e-02],
        [-1.20237991e-02, -1.32681133e-04, -5.1

In [25]:
#!g1.1
sample_hidden

<tf.Tensor: shape=(32, 300), dtype=float32, numpy=
array([[-0.00057022, -0.00643045, -0.00013946, ..., -0.00146251,
         0.00561243,  0.01890766],
       [-0.00057022, -0.00643045, -0.00013946, ..., -0.00146251,
         0.00561242,  0.01890766],
       [-0.00057021, -0.00643045, -0.00013946, ..., -0.00146251,
         0.00561242,  0.01890766],
       ...,
       [-0.00057023, -0.00643045, -0.00013945, ..., -0.0014625 ,
         0.00561243,  0.01890766],
       [ 0.01085338,  0.00407551,  0.0053734 , ...,  0.00477865,
         0.00355481, -0.00346712],
       [-0.00057021, -0.00643045, -0.00013946, ..., -0.00146251,
         0.00561242,  0.01890766]], dtype=float32)>

In [26]:
#!g1.1
# Компиляция модели
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    # определяем ненулевые элементы
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    #переводим mask к типу loss_
    mask = tf.cast(mask, dtype=loss_.dtype)
    #умножаем loss на маску
    loss_ *= mask

    return tf.reduce_mean(loss_) # возвращаем среднее значение элементов тензора

In [27]:
#!g1.1
# Обучение модели
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([sum_wordindex['bos']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [28]:
#!g1.1
checkpoint_dir = './training_summ_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [29]:
#!g1.1
EPOCHS = 100

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 10 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

2023-03-05 16:55:19.492431: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-03-05 16:55:23.376507: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "Tesla V100-PCIE-32GB" frequency: 1380 num_cores: 80 environment { key: "architecture" value: "7.0" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 98304 memory_size: 32501137408 bandwidth: 898048000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-03-05 16:55:23.377137: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "

Epoch 1 Batch 0 Loss 0.8940
Epoch 1 Batch 10 Loss 0.8960
Epoch 1 Batch 20 Loss 0.8834
Epoch 1 Batch 30 Loss 1.0181
Epoch 1 Loss 0.8942
Time taken for 1 epoch 69.05212545394897 sec

Epoch 2 Batch 0 Loss 0.8127
Epoch 2 Batch 10 Loss 0.8725
Epoch 2 Batch 20 Loss 0.8227
Epoch 2 Batch 30 Loss 0.8716
Epoch 2 Loss 0.8268
Time taken for 1 epoch 7.493474721908569 sec

Epoch 3 Batch 0 Loss 0.9115
Epoch 3 Batch 10 Loss 0.8475
Epoch 3 Batch 20 Loss 0.8595
Epoch 3 Batch 30 Loss 0.7972
Epoch 3 Loss 0.7970
Time taken for 1 epoch 5.701946020126343 sec

Epoch 4 Batch 0 Loss 0.8034
Epoch 4 Batch 10 Loss 0.7740
Epoch 4 Batch 20 Loss 0.7666
Epoch 4 Batch 30 Loss 0.7008
Epoch 4 Loss 0.7704
Time taken for 1 epoch 7.506965398788452 sec

Epoch 5 Batch 0 Loss 0.7497
Epoch 5 Batch 10 Loss 0.8201
Epoch 5 Batch 20 Loss 0.7870
Epoch 5 Batch 30 Loss 0.7917
Epoch 5 Loss 0.7568
Time taken for 1 epoch 5.671932220458984 sec

Epoch 6 Batch 0 Loss 0.7568
Epoch 6 Batch 10 Loss 0.7143
Epoch 6 Batch 20 Loss 0.6366
Epoch 6 B

In [30]:
#!g1.1
# Оценка модели
def evaluate(sentence):
    inputs = [tok_text.word_index[i] for i in sentence.split(' ') if i !='']
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_len_text,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, latent_dim))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tok_sum.word_index['bos']], 0)

    for t in range(max_len_sum):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += tok_sum.index_word[predicted_id] + ' '

        if tok_sum.index_word[predicted_id] == 'eos':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [31]:
#!g1.1
def summ(sentence):
    result, sentence = evaluate(sentence)

    return result

In [32]:
#!g1.1
# Функция перевода последовательности индексов в текст
def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_text_index[i]+' '
    return newString
# Функция перевода последовательности индексов в title
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=sum_wordindex['bos']) and i!=sum_wordindex['eos']):
            newString=newString+reverse_sum_index[i]+' '
    return newString

In [33]:
#!g1.1
for i in range(5):
    text2 = seq2text(padded_x_test[i])
    print("Original title:", seq2summary(padded_x_test_sum[i]))
    print("Predicted title: ", summ(text2.strip()))
    print("\n")

Original title: или куда 
Predicted title:  «на костылях по монако» новое несчастье с головиным eos 


Original title: «люди в 
Predicted title:  оставят без «стольника» eos 


Original title: ссср как выборы 
Predicted title:  16 ронинов кремль изучает список «бизнесменов возвращенцев» eos 


Original title: «он с 
Predicted title:  «он заставил уважать америку» eos 


Original title: почему не пенсии 
Predicted title:  норвегия признала терроризм eos 




**Вывод:** результаты модели плохие